Questa è una implementazione non ufficiale del paper **"Ladder Variational Autoencoders"**  di *Casper Kaae Sønderby, Tapani Raiko, Lars Maaløe, Søren Kaae Sønderby, Ole Winther*
(https://arxiv.org/pdf/1602.02282v3.pdf)

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.datasets import load_digits
from torchvision.datasets import FashionMNIST
from torch.utils.data import Subset, DataLoader
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.multivariate_normal import MultivariateNormal
import math
from torchvision import transforms
from torch.linalg import multi_dot
import gc

# Init



In [2]:

# Controlla la disponibilità della GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # Imposta il dispositivo sulla GPU
else:
    device = torch.device("cpu")  # Se la GPU non è disponibile, utilizza la CPU



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#ink originale: https://drive.google.com/file/d/0B7EVK8r0v71pZjFTYXZWM3FlRnM/view?usp=drive_link&resourcekey=0-dYn9z10tMJOBAkviAcfdyQ
#spostatelo nel vostro drive e copiatelo in locale (perchè il dataloader carica più velocemente le immagini leggendo da qui che dal vostro drive)
!cp '/content/drive/MyDrive/Generative_AI/datasets/celebA/img_align_celeba.zip' celebA.zip

In [5]:
import zipfile

with zipfile.ZipFile("celebA.zip", 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

# Model

Blocco che serve a approssimare le distribuzioni q e p

In [20]:
'''
    Tale blocco prende z_i in ingresso (oppure nel primo caso x) e cerca di approssimare
    q(z_i+1| z_i) ( o q(z_1| x) nel primo caso). Nel paper tali distribuzione sono
    delle Multivariate dove la media viene trovata come:

                        mean = Linear(MLP(z_i))

    mentre la varianza (elementi nella matrice di covarianza diagonale) come:

                        cov = Softplus(Linear(MLP(z_i)))

    NB: L'MLP è CONDIVISO tra i due

    Tale blocco è usato anche nel ramo del DECODER in quanto da paper le distribuzioni
    sono parametrizzate con la stessa logica

'''

# class Block(nn.Module):
#   def __init__(self, input_shape, latent_space_dimension, hidden_neurons):
#     super(Block, self).__init__()

#     #MLP(z_i)
#     self.MLP = nn.Sequential( nn.Linear(input_shape,8*hidden_neurons),
#                               nn.BatchNorm1d(8*hidden_neurons),
#                               nn.ReLU(),
#                               nn.Linear(8*hidden_neurons,4*hidden_neurons),
#                               nn.BatchNorm1d(4*hidden_neurons),
#                               nn.ReLU(),
#                               nn.Linear(4*hidden_neurons,2*hidden_neurons),
#                               nn.BatchNorm1d(2*hidden_neurons),
#                               nn.ReLU(),
#                               nn.Linear(2*hidden_neurons,hidden_neurons)
#                               )

#     self.mean_net = nn.Sequential(
#                               #Linear(MLP(z_i))
#                               nn.Linear(hidden_neurons, latent_space_dimension),
#                               nn.BatchNorm1d(latent_space_dimension),
#                               nn.LeakyReLU()
#                             )

#     self.log_std_net = nn.Sequential(
#                             #Linear(MLP(z_i))
#                             nn.Linear(hidden_neurons, latent_space_dimension),
#                             nn.BatchNorm1d(latent_space_dimension),

#                             #Softplus(Linear(MLP(z_i)))
#                             nn.Softplus()
#                             )

#   #input può essere x se è il primo blocco, z_i altrimenti (SE ENCODER)
#   #input può essere z4 se è il primo blocco, z_i altrimenti (SE DECODER)
#   def forward(self, input):

#       d = self.MLP(input)

#       mean = self.mean_net(d)

#       log_std = self.log_std_net(d)

#       return [mean, log_std]

from torch.nn.modules.activation import LeakyReLU
'''
    Tale blocco prende z_i in ingresso (oppure nel primo caso x) e cerca di approssimare
    q(z_i+1| z_i) ( o q(z_1| x) nel primo caso). Nel paper tali distribuzione sono
    delle Multivariate dove la media viene trovata come:

                        mean = Linear(MLP(z_i))

    mentre la varianza (elementi nella matrice di covarianza diagonale) come:

                        cov = Softplus(Linear(MLP(z_i)))

    NB: L'MLP è CONDIVISO tra i due

    Tale blocco è usato anche nel ramo del DECODER in quanto da paper le distribuzioni
    sono parametrizzate con la stessa logica

'''

class Block(nn.Module):
  def __init__(self, input_shape, latent_space_dimension, hidden_neurons):
    super(Block, self).__init__()

    #MLP(z_i)
    # self.MLP = nn.Sequential( nn.Linear(input_shape,8*hidden_neurons),
    #                           nn.BatchNorm1d(8*hidden_neurons),
    #                           nn.ReLU(),
    #                           nn.Linear(8*hidden_neurons,6*hidden_neurons),
    #                           nn.BatchNorm1d(6*hidden_neurons),
    #                           nn.Tanh(),
    #                           nn.Linear(6*hidden_neurons,4*hidden_neurons),
    #                           nn.BatchNorm1d(4*hidden_neurons),
    #                           nn.ReLU(),
    #                           nn.Linear(4*hidden_neurons,2*hidden_neurons),
    #                           nn.BatchNorm1d(2*hidden_neurons),
    #                           nn.ReLU(),
    #                           nn.Linear(2*hidden_neurons,input_shape),
    #                           nn.LeakyReLU()
    #                           )

    out_channels = 32
    kernel_size = 4
    self.MLP = nn.Sequential(
                              #nn.BatchNorm1d(input_shape),

                              nn.Conv2d(1,out_channels,kernel_size, stride=1, padding="same"),
                              nn.BatchNorm2d(out_channels),
                              nn.ELU(),

                              nn.Conv2d(out_channels,out_channels,kernel_size, stride=1,padding="same"),
                              nn.BatchNorm2d(out_channels),
                              nn.ELU(),

                              nn.Conv2d(out_channels,2*out_channels,kernel_size, stride=1,padding="same"),
                              nn.BatchNorm2d(2*out_channels),
                              nn.ELU(),

                              nn.Conv2d(2*out_channels,2*out_channels,kernel_size, stride=1,padding="same"),
                              nn.BatchNorm2d(2*out_channels),
                              nn.ELU(),

                              nn.Flatten(),
                              nn.LazyLinear(input_shape)

    )

    self.mean_net = nn.Sequential(
                              #Linear(MLP(z_i))
                              nn.Linear(input_shape, input_shape),
                              nn.BatchNorm1d(input_shape),
                              nn.Linear(input_shape, latent_space_dimension),
                              nn.BatchNorm1d(latent_space_dimension),
                              nn.LeakyReLU()
                            )

    self.log_std_net = nn.Sequential(
                            #Linear(MLP(z_i))
                            nn.Linear(input_shape, input_shape),
                            nn.BatchNorm1d(input_shape),
                            nn.Linear(input_shape, latent_space_dimension),
                            nn.BatchNorm1d(latent_space_dimension),

                            #Softplus(Linear(MLP(z_i)))
                            nn.Softplus()
                            )

  #input può essere x se è il primo blocco, z_i altrimenti (SE ENCODER)
  #input può essere z4 se è il primo blocco, z_i altrimenti (SE DECODER)
  def forward(self, input):

      #da (N,W*H) a (N,1,W,H)
      input = torch.reshape(input, (input.shape[0],1,int(math.sqrt(input.shape[1])),int(math.sqrt(input.shape[1])) ))

      d = self.MLP(input)
      d = d+torch.flatten(input,2,3).squeeze(1)

      mean = self.mean_net(d)
      log_std = self.log_std_net(d)

      return [mean, log_std]

Encoder

In [21]:
class Encoder(nn.Module):
  def __init__(self, input_shape_image, latent_space_dimension, hidden_neurons,possible_pixel_values):
    super(Encoder, self).__init__()

    self.possible_pixel_values = possible_pixel_values

    #creo blocco che approssimerà q(z1|x)
    self.q_z1_x = Block(input_shape_image, latent_space_dimension, hidden_neurons)

    #creo funzione che combina Linear([x,z1])
    self.combine_z1_x = nn.Linear(input_shape_image+latent_space_dimension, latent_space_dimension)

    #creo blocco che approssimerà q(z2|z1,x)
    self.q_z2_z1_x = Block(latent_space_dimension, latent_space_dimension, hidden_neurons)

    #creo funzione che combina Linear([x,z1,z2])
    self.combine_z2_z1_x = nn.Linear(input_shape_image+latent_space_dimension*2, latent_space_dimension)

    #creo blocco che approssimerà q(z3|z2,z1,x)
    self.q_z3_z2_z1_x = Block(latent_space_dimension, latent_space_dimension, hidden_neurons)

    #creo funzione che combina Linear([x,z1,z2,z3])
    self.combine_z3_z2_z1_x = nn.Linear(input_shape_image+latent_space_dimension*3, latent_space_dimension)

    #creo blocco che approssimerà q(z4|z3,z2,z1,x)
    self.q_z4_z3_z2_z1_x = Block(latent_space_dimension, latent_space_dimension, hidden_neurons)

  def forward(self, x):

    #normalizzo il batch
    x_norm = x/(self.possible_pixel_values-1)

    #flatto   da (N,W,H) a (N,W*H)
    x = torch.flatten(x,start_dim=1, end_dim=2)


    #:::::::::::::::::::::::: PARTE INFERENZIALE ::::::::::::::::::::::::::::::::::::::::::

    #_____ q(z1|x)________________________________
    #ottengo parametri della distribuzione in funzione dell'input x
    [mean, log_std] = self.q_z1_x(x)
    #creo la distribuzione
    q = MultivariateNormal(mean, torch.diag_embed( torch.clamp( torch.exp(log_std),min=1e-8)).to(device))
    #campiono z1 col reparametrization trick
    z1 = q.rsample()
    #calcolo ln(q(z1|x))
    ln_q_z1_x = q.log_prob(z1)


    #_____ q(z2|z1,x)________________________________
    #dato che la distribuzione dipende sia da z1 che da x, creo un singolo input che è in funzione dei due
    #(N, input_shape+latent_dimension)
    input = torch.concat( (z1,x), dim=-1)
    #(N, latent_dimension)
    input = self.combine_z1_x(input)
    #ottengo parametri della distribuzione in funzione dell'input z1 e x
    [mean, log_std] = self.q_z2_z1_x(input)
    #creo la distribuzione
    q = MultivariateNormal(mean, torch.diag_embed( torch.clamp( torch.exp(log_std),min=1e-8)).to(device))
    #campiono z2 col reparametrization trick
    z2 = q.rsample()

    #calcolo ln(q(z2|z1,x))
    ln_q_z2_z1_x = q.log_prob(z2)

    #_____ q(z3|z2,z1,x)________________________________
    #dato che la distribuzione dipende sia da z2,z1 che da x, creo un singolo input che è in funzione dei tre
    #(N, input_shape+latent_dimension*2)
    input = torch.concat( (z2,z1,x), dim=-1)
    #(N, latent_dimension)
    input = self.combine_z2_z1_x(input)
    #ottengo parametri della distribuzione in funzione dell'input z2,z1 e x
    [mean, log_std] = self.q_z3_z2_z1_x(input)
    #creo la distribuzione
    q = MultivariateNormal(mean, torch.diag_embed( torch.clamp( torch.exp(log_std),min=1e-8)).to(device))
    #campiono z3 col reparametrization trick
    z3 = q.rsample()
    #calcolo ln(q(z3|z2,z1,x))
    ln_q_z3_z2_z1_x = q.log_prob(z3)

    #_____ q(z4|z3,z2,z1,x)________________________________
    #dato che la distribuzione dipende sia da z3,z2,z1 che da x, creo un singolo input che è in funzione dei quattro
    #(N, input_shape+latent_dimension*3)
    input = torch.concat( (z3,z2,z1,x), dim=-1)
    #(N, latent_dimension)
    input = self.combine_z3_z2_z1_x(input)
    #ottengo parametri della distribuzione in funzione dell'input z3,z2,z1 e x
    [mean, log_std] = self.q_z4_z3_z2_z1_x(input)
    #creo la distribuzione
    q = MultivariateNormal(mean, torch.diag_embed( torch.clamp( torch.exp(log_std),min=1e-8)).to(device))
    #campiono z4 col reparametrization trick
    z4 = q.rsample()
    #calcolo ln(q(z4|z3,z2,z1,x))
    ln_q_z4_z3_z2_z1_x = q.log_prob(z4)

    #ritorno gli z campionati e i loro logaritmi
    return z1,z2,z3,z4, ln_q_z1_x, ln_q_z2_z1_x, ln_q_z3_z2_z1_x, ln_q_z4_z3_z2_z1_x



Decoder

In [22]:
class Decoder(nn.Module):
  def __init__(self, input_shape_image, latent_space_dimension, hidden_neurons,possible_pixel_values):
    super(Decoder, self).__init__()

    self.input_shape_image = input_shape_image
    self.possible_pixel_values = possible_pixel_values

    #prior p(z4)=N(0,I)
    self.p_z4 = MultivariateNormal(torch.zeros(latent_space_dimension).to(device), torch.eye(latent_space_dimension).to(device))

    self.p_z3_z4 =  Block(latent_space_dimension, latent_space_dimension, hidden_neurons)

    self.p_z2_z3 =  Block(latent_space_dimension, latent_space_dimension, hidden_neurons)

    self.p_z1_z2 =  Block(latent_space_dimension, latent_space_dimension, hidden_neurons)

    self.p_x_z1 = nn.Sequential(nn.Linear(latent_space_dimension, hidden_neurons),
                                nn.ReLU(),
                                nn.BatchNorm1d(hidden_neurons),
                                nn.Linear(hidden_neurons,2*hidden_neurons),
                                nn.ReLU(),
                                nn.BatchNorm1d(2*hidden_neurons),
                                nn.Linear(2*hidden_neurons,2*hidden_neurons),
                                nn.ReLU(),
                                #produco media e log std per z1
                                nn.Linear(2*hidden_neurons, input_shape_image*possible_pixel_values),
                                nn.LeakyReLU()
                                )


  def sample(self):

    #campiono dalla prior p(z_4)=N(0,I)
    z4 = self.p_z4.sample()
    z4 = z4.unsqueeze(0)

    #lo utilizzo per calcolarmi i parametri di p(z3|z4) e campionare z3
    [mean, log_std] = self.p_z3_z4(z4)
    z3 = MultivariateNormal(mean, torch.diag_embed( torch.clamp( torch.exp(log_std),min=1e-8))).sample()
    #lo utilizzo per calcolarmi i parametri di p(z2|z3) e campionare z2
    [mean, log_std] = self.p_z2_z3(z3)
    z2 = MultivariateNormal(mean, torch.diag_embed( torch.clamp( torch.exp(log_std),min=1e-8))).sample()
    #lo utilizzo per calcolarmi i parametri di p(z1|z2) e campionare z1
    [mean, log_std] = self.p_z1_z2(z2)
    z1 = MultivariateNormal(mean, torch.diag_embed( torch.clamp( torch.exp(log_std),min=1e-8))).sample()

    #calcolo p(x|1)
    output = self.p_x_z1(z1)

    #reshape in (1, input_shape, possible_pixel_values)
    logits = output.reshape(1,self.input_shape_image, self.possible_pixel_values)

    #eseguo la softmax sull'ultimo livello
    probabilities = torch.softmax(logits,dim=-1)

    probabilities = probabilities.view(-1, self.possible_pixel_values)


    sample = torch.multinomial(probabilities, num_samples=1)

    x = sample.view(self.input_shape_image)
    return x


  def forward(self, z1,z2,z3,z4,x):

    #calcolo ln(p(z4)) usando la prior N(0,I)
    ln_p_z4 = self.p_z4.log_prob(z4)

    #_____ p(z3|z4)________________________________
    #calcolo i parametri della distribuzione p in funzione di z4
    [mean, log_std] = self.p_z3_z4(z4)
    #creo la distribuzione
    q = MultivariateNormal(mean, torch.diag_embed( torch.clamp( torch.exp(log_std),min=1e-8)).to(device))
    #valuto z3 campionato nella fase inferenziale con la prior
    ln_p_z3_z4 = q.log_prob(z3)

    #_____ p(z2|z3)________________________________
    #calcolo i parametri della distribuzione p in funzione di z3
    [mean, log_std] = self.p_z2_z3(z3)
    #creo la distribuzione
    q = MultivariateNormal(mean, torch.diag_embed( torch.clamp( torch.exp(log_std),min=1e-8)).to(device))
    #valuto z2 campionato nella fase inferenziale con la prior
    ln_p_z2_z3 = q.log_prob(z2)

    #_____ p(z1|z2)________________________________
    #calcolo i parametri della distribuzione p in funzione di z2
    [mean, log_std] = self.p_z1_z2(z2)
    #creo la distribuzione
    q = MultivariateNormal(mean, torch.diag_embed( torch.clamp( torch.exp(log_std),min=1e-8)).to(device))
    #valuto z1 campionato nella fase inferenziale con la prior
    ln_p_z1_z2 = q.log_prob(z1)

    #_____Ricostruzione dell'input x_______________
    x = torch.flatten(x, start_dim=1, end_dim=2)

    #utilizzo la rete per ricostruire l'immagine (N, input_shape*possible_pixel_values)
    output = self.p_x_z1(z1)

    #reshape in (N, input_shape, possible_pixel_values)
    logits = output.reshape(output.shape[0],self.input_shape_image, self.possible_pixel_values)

    #eseguo la softmax sull'ultimo livello
    output_probabilities = torch.softmax(logits,dim=-1)
    EPS = 1.e-5
    output_probabilities = torch.clamp(output_probabilities, EPS,1. - EPS)

    #li converto in logaritmi
    log_probabilities = torch.log(output_probabilities)

    #trasformo x in one hot encoding
    x_one_hot = F.one_hot(x.long(),num_classes=self.possible_pixel_values)

    #li utilizzo per azzerare la probabilità relativa al valore del pixel
    selected_probabilities = x_one_hot*log_probabilities

    #li sommo (N,), ossia ogni cella contiene la somma dei ln(p(x|z1)) per gli N x
    #Questo sarebbe il Reconstruction Error
    ln_p_x_z1 = selected_probabilities.sum(-1).sum(-1)

    #ritorno il reconstruction error e tutte le p(zi|zi+1)
    return ln_p_x_z1, ln_p_z1_z2, ln_p_z2_z3, ln_p_z3_z4, ln_p_z4



Ladder VAE

In [23]:
class Ladder_VAE(nn.Module):
  def __init__(self, input_shape_image, latent_space_dimension, hidden_neurons, possible_pixel_values):
    super(Ladder_VAE, self).__init__()

    self.encoder = Encoder(input_shape_image,latent_space_dimension,hidden_neurons,possible_pixel_values)

    self.decoder = Decoder(input_shape_image,latent_space_dimension,hidden_neurons,possible_pixel_values)


  def sample(self):
    return self.decoder.sample()

  def forward(self,x):

    #do all'encoder l'immagine x e ottengo z1,z2,z3,z4 e i rispettivi logaritmi
    z1,z2,z3,z4, ln_q_z1_x, ln_q_z2_z1_x, ln_q_z3_z2_z1_x, ln_q_z4_z3_z2_z1_x = self.encoder(x)

    #do al decoder i vari z per calcolarne la log prob con le loro prior e ottenere il reconstruction error
    RE,ln_p_z1_z2, ln_p_z2_z3, ln_p_z3_z4, ln_p_z4 = self.decoder(z1,z2,z3,z4,x)

    '''
      Avendo 4 spazi latenti allora l'ELBO sarà:
        ELBO = RE - KL1 - KL2 - KL3 - KL4

      dove:

        RE = ln(p(x|z1))  --> ritornato solo dall' decoder

        KL1 = ln( q(z1|x)/p(z1|z2) )  --> q ritornato dall'encoder, p dal decoder

        KLi = ln( q(zi|z_<i , x)/p(zi|z_i+1))   per i=2,3,4

      NB: Sarebbe opportuno valutare l'expected value con (per esempio) un Monte-Carlo
      approach ma qui per semplicità l'ho omesso (quindi solo un campione per approssimare
      ciascune expected value). Guardare i primi VAE che ho implementato se si volesse
      avere una idea su come implementare Monte-Carlo.

      NB: esiste una versione successiva in cui calcolo la KL in maniera precisa grazie a un teorema

    '''

    KL1 = ln_q_z1_x - ln_p_z1_z2
    KL2 = ln_q_z2_z1_x - ln_p_z2_z3
    KL3 = ln_q_z3_z2_z1_x - ln_p_z3_z4
    KL4 = ln_q_z4_z3_z2_z1_x - ln_p_z4

    ELBO = (RE - KL1 - KL2 - KL3 - KL4).mean()

    return -ELBO

# MAIN

In [ ]:
##################################### GPU + Path ##########################################

# Controlla la disponibilità della GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # Imposta il dispositivo sulla GPU
else:
    device = torch.device("cpu")  # Se la GPU non è disponibile, utilizza la CPU

print("Device utilizzato:", device)
print("Numero di GPU disponibili:", torch.cuda.device_count())


#path dove salvare il modello migliore e i vari output di ogni epoca valida
path_to_model = "/content/drive/MyDrive/Generative_AI/datasets/celebA/model/model_Ladder_Hierarchical_VAE.pth"
path_to_output = "/content/drive/MyDrive/Generative_AI/datasets/celebA/output/Ladder_Hierarchical_VAE_"





##################################### Dataloader ##########################################

#per motivi di efficienza, scegliere il rescaling e il massimo valore che ogni pixel può assumere
resize_to = 45
max_pixel_value = 30

input_shape_image = resize_to*resize_to
possible_pixel_values = max_pixel_value+1


def load_data():


    # Definisci le trasformazioni da applicare alle immagini durante il caricamento
    transform = transforms.Compose([
        transforms.Resize( (resize_to, resize_to) ), #rescaling di ogni immagine
        transforms.Grayscale(),  # Trasforma l'immagine in bianco e nero
        transforms.ToTensor(),# Converte l'immagine in un tensore
        transforms.Lambda(lambda x: torch.round(x*(max_pixel_value))), #normalizzo i valori dei pixel e li forzo ad essere interi
        #transforms.Lambda(lambda x: x.to(torch.float32))
    ])

    # Crea un oggetto ImageFolder per caricare le immagini dalla cartella specificata e applica le trasformazioni definite
    dataset = datasets.ImageFolder('/content/dataset/', transform=transform)

    # Calcola l'indice per dividere il dataset tra training set e validation set
    split_ratio = 0.8  # Ratio di suddivisione (80% per il training set, 20% per il validation set)
    dataset_size = len(dataset)
    split_index = int(split_ratio * dataset_size)

    # Crea due sottoinsiemi distinti per il training set e il validation set
    train_dataset = Subset(dataset, range(0, split_index))
    val_dataset = Subset(dataset, range(split_index, dataset_size))

    # Crea i DataLoader per il training set e il validation set
    batch_size = 32
    training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return training_loader, validation_loader










################################### Training + validation #####################################

def train_model_on_given_gpu():

    #definisco la dimensione dello spazio latente
    latent_space_dimension = 36 #deve essere intera la sua radice quadrata
    #nuumero di hidden neurons nell'encoder e decoder
    hidden_neurons = 64


    #---- creazione del modello

    model = Ladder_VAE(input_shape_image, latent_space_dimension, hidden_neurons,possible_pixel_values)
    model.to(device)

    # Esegui un passaggio di inoltro dummy per il LazyLinear
    inputs = torch.round(torch.rand((2, resize_to, resize_to),dtype=torch.float32)*max_pixel_value).to(device)
    print("inputs", inputs.shape)
    dummy_output = model(inputs).item()

    print("Numero parametri modello: ",sum(p.numel() for p in model.parameters() if p.requires_grad))

    #parametri per il learning
    learning_rate = 1e-3

    parameters_to_optimize = [p for p in model.parameters() if p.requires_grad == True]

    optimizer = torch.optim.Adamax(parameters_to_optimize, lr=learning_rate)


    #------ Funzione per salvare una griglia di campioni decodificati dallo spazio latente ogni volta che la validation è migliore
    def sample_and_save(model, name, input_shape):

      model.eval()

      #voglio campionare 16 immagini e le voglio in una griglia 4x4
      n=4
      number_of_grid_cells = n*n
      #quindi dico al modello di campionarmi 16 immagini
      xs = np.zeros((number_of_grid_cells,input_shape))
      for i in np.arange(number_of_grid_cells):
        generated_sample = model.sample().cpu() # il .module serve per andare oltre il wrapping di DataParallel
        #lo stacco dal grafo di computazione
        generated_sample = generated_sample.detach().numpy()
        xs[i,:] = generated_sample


      fig, ax = plt.subplots(n, n)
      for i, ax in enumerate(ax.flatten()):
          plottable_image = np.reshape(xs[i], (int(math.sqrt(input_shape)), int(math.sqrt(input_shape))))
          ax.imshow(plottable_image, cmap='gray')
          ax.axis('off')

      plt.savefig(path_to_output+'epoca_' +str(name)+ '.pdf', bbox_inches='tight')
      plt.close()



    #---- Training e validation
    number_of_epochs = 1000
    #fisso il limite massimo di batch di training e validazione
    max_batch_for_training = 800
    max_batch_for_validation = 170


    #qui salvo il migliore modello, ossia quello che ha la loss sulla validazione migliore
    best_model = model
    best_validation_loss = 1000000

    patience = 0
    max_patience = 30

    training_loader, validation_loader = load_data()

    grd_acc = 1

    for epoch in range(number_of_epochs):
      model.train()
      print("Epoca "+str(epoch)+" _____________________________________________________________________")

      num_batch = 1
      for batch, _ in training_loader:

        #reshaping di ogni batch da (N, 1, W, H) a (N, W, H)
        batch = batch.squeeze(1)

        batch = batch.to(device)

        loss = model.forward(batch)

        del batch
        gc.collect()
        torch.cuda.empty_cache()

        #calcolo le derivate parziali della loss rispetto ogni parametro NB: LA mean() E' PERCHE' UTILIZZO N GPU E CIASCUNA RITORNA LA SUA LOSS
        (loss.mean()/grd_acc).backward(retain_graph=True)
        torch.cuda.empty_cache()

        #se ho accumulato il gradiente di un numero sufficiente di batch, allora backpropago
        if ( (num_batch % grd_acc) == 0):
            #adesso ogni parametro ha in .grad il gradiente. Aggiorno il suo valore
            optimizer.step()

            #resetto il .grad di ogni parametro (altrimenti sommo quello attuale al successivo che calcoleremo nell'epoca dopo)
            optimizer.zero_grad()

        print("   Loss batch: ",str(num_batch),": ", loss, "          Memoria GPU utilizzata  -> ", round(torch.cuda.memory_allocated()*(1e-9),4), "GB")

        #se ho superato il numero massimo di batch per il training, esco
        if num_batch >= max_batch_for_training:
            break
        else:
            num_batch = num_batch + 1

      #alla fine di ogni epoca, valuto come si comporta la loss col validation set
      print("   ___________________________")
      model.eval()
      validation_loss = 0
      N = 0

      torch.cuda.empty_cache()

      num_batch = 1
      for batch, _ in validation_loader:

        batch = batch.squeeze(1)
        batch = batch.to(device)
        #batch = batch.to(torch.float32)
        loss_i = model.forward(batch)
        validation_loss = validation_loss + loss_i.mean().item()# NB: .mean() SOLO PERCH' UTILIZZO N GPU E QUINDI VOGLIO LA MEDIA DI OGNI LOSS RITORNATA DA OGNI GPU
        N = N +  1

        del batch
        gc.collect()
        torch.cuda.empty_cache()

        print("   Loss validation batch ",str(num_batch),": ",loss_i)
        #se ho superato il numero massimo di batch per il validation, esco
        if num_batch >= max_batch_for_validation:
            break
        else:
            num_batch = num_batch + 1

        del loss_i

      validation_loss = validation_loss/N
      print("   Loss media validation: ",str(validation_loss))

      #se tale modello ha una loss migliore di quella attualmente migliore..
      if validation_loss < best_validation_loss:
        patience = 0
        best_validation_loss = validation_loss
        print("   la loss risulta essere migliore")
        torch.save(model.state_dict(), path_to_model)
        #campiono e salvo
        sample_and_save(model, epoch, input_shape_image)
      else:
        print("   patience= "+ str(patience+1))
        patience = patience + 1

      if patience > max_patience:
        print("")
        print("Patience massimo superato. Fine del training")
        break

      del validation_loss



if __name__=="__main__":

    train_model_on_given_gpu()